<a href="https://colab.research.google.com/github/sierrarowley/research_fall2020/blob/master/prov_crimedata_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import files, drive
import pandas as pd 
import io 
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

drive.mount('drive')
uploaded2 = files.upload()
providence_pd = pd.read_csv(io.BytesIO(uploaded2['Providence_Police_log2.csv']))

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


Saving Providence_Police_log2.csv to Providence_Police_log2 (2).csv


In [14]:
crimesmap = np.array(providence_pd)
# select address column from providence data set, plus other column to be overwritten with long/lat
crimesmap_ll = crimesmap[:, 1:3]
locator = Nominatim(user_agent='myGeocoder')

# select time and type of crime columns from data
crimesmap_tt = np.column_stack((crimesmap[:, 2], crimesmap[:, 5]))

# give crime number for each type of crime
crime_type = set(crimesmap[:, 5])
type_map = {i: n for n,i in enumerate(crime_type)}

row = crimesmap_ll.shape[0] - 1
while row >= 0:
  # turn address into lattitude and longitude
  location = None
  # some of the addresses are type float, these rows will be deleted
  if type(crimesmap_ll[row, 0]) == type(', Providence, RI') and '&' not in crimesmap_ll[row, 0]:
    address = crimesmap_ll[row, 0] + ', Providence, RI'
    try:
      location = locator.geocode(address)
    except GeocoderTimedOut as e:
      location = None
  # make sure lat and long are actually in providence
  if location != None and int(location.latitude) == 41 and int(location.longitude) == -71:
    crimesmap_ll[row, 0] = location.latitude
    crimesmap_ll[row, 1] = location.longitude
  # if address not found, just skip row (delete it)
  else:
    crimesmap_ll = np.delete(crimesmap_ll, row, axis=0)

  # set crime type at this row to be the corresponding number
  crimesmap_tt[row, 1] = type_map[crimesmap_tt[row, 1]]
  # isolate time from date of occurance
  crimesmap_tt[row, 0] = int(crimesmap_tt[row, 0][11: 13])

  row -= 1

dftt = pd.DataFrame(data=crimesmap_tt)
dfll = pd.DataFrame(data=crimesmap_ll)
dftt.to_csv('prov_data_tt.csv')
dfll.to_csv('prov_data_ll.csv')
!cp prov_data_tt.csv "drive/My Drive/"
!cp prov_data_ll.csv "drive/My Drive/"